In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cProfile

### Dokładność maszynowa komputera

In [3]:
epsilon = 1

while 1+epsilon != 1:
    epsilon *= 0.5

print(epsilon*2)
print(np.finfo(float).eps)

2.220446049250313e-16
2.220446049250313e-16


### Schemat Hornera

In [4]:
def eval_pol(x, coeff):
    p = coeff[-1]
    n = len(coeff)
    for i in range(1, n):
        p = coeff[n-i-1] + p*x
    return p

## Rozwiązywanie układów równań:

### 1. Metoda eliminacji Gaussa

In [5]:
def partial_elimination(A,b):
    A = np.array(A).astype(float)
    b = np.array(b).astype(float)
    n = len(b)
    row_idx = 0
    col_idx = 0
    for i in range(n-1):
        col_nums = []
        col_nums.append(A[row_idx:,i])
        col_max = np.max(col_nums)
        pivot_row = np.argwhere(A[:,col_idx]==col_max)[-1][0]
        A[[row_idx, pivot_row]] = A[[pivot_row, row_idx]]
        b[[row_idx, pivot_row]] = b[[pivot_row, row_idx]]
        for k in range(row_idx, n-1):
            lam = A[k+1,col_idx]/A[row_idx,col_idx]
            A[k+1, :] = A[k+1, :] - lam*A[row_idx, :]
            b[k+1] = b[k+1] - lam*b[row_idx]
        row_idx += 1
        col_idx += 1  
    for k in range(n-1,-1,-1):
        b[k] = (b[k] - np.dot(A[k,k+ 1:n],b[k+ 1:n]))/A[k,k] 
    return A, b  

### 2. Metoda Gaussa-Seidla

In [6]:
def GaussSeidel(A,b,n):
    L = np.tril(A, k=-1)
    U = np.triu(A, k=1)
    D = np.zeros((len(A), len(A)), float)
    np.fill_diagonal(D, np.diag(A))
    x = np.zeros(len(A))
    T = -(np.linalg.inv(D+L))@U
    C = (np.linalg.inv(D+L))@b
    xs = []
    for i in range(n):
        x = T@x+C
        xs.append(x)
    return x, xs

## Rozwiązywanie równań

### 1. Bisekcja

In [7]:
def bisection(f, a, b):
    i = 0
    flops = 3
    fa = f(a)
    fb = f(b)
    if fa == 0.0:
        return a
    if fb == 0.0:
        return b
    m = a + (b-a)/2
    fm = f(m)
    while not np.isclose(fm, 0):
        if (abs(fm) > abs(fa)) and (abs(fm)>abs(fb)):
            print(f'W przedziale ({a}, {b}) występuje osobliwość')
            return None
        if (fa*fm>0) and (fb*fm>0):
            print(f'W przedziale ({a}, {b}) nie występuje pierwiastek równiania.')
            return None
        if fm*fb<0:
            a, fa = m, fm
            m = a + (b-a)/2
            fm = f(m)
        if fa*fm<0:
            b, fb = m, fm
            m = a + (b-a)/2
            fm = f(m)
        i += 1
        flops += 3
    return m, i, flops

### 2. Metoda Brenta

In [8]:
def brent(f,a,b):
    i = 0
    flops = 22
    c = (a+b)/2
    fa = f(a)
    fb = f(b)
    fc = f(c)
    x = -(a*fb*fc*(fb-fc)+b*fc*fa*(fc-fa)+c*fa*fb*(fa-fb))/((fa-fb)*(fb-fc)*(fc-fa))
    fx = f(x)
    while not np.isclose(fx, 0):
        if fa*fc<0:
            b = c
            c = (a+b)/2
            fa = f(a)
            fb = f(b)
            fc = f(c)
            x = -(a*fb*fc*(fb-fc)+b*fc*fa*(fc-fa)+c*fa*fb*(fa-fb))/((fa-fb)*(fb-fc)*(fc-fa))
            fx = f(x)
        if fc*fb<0:
            a = c
            c = (a+b)/2
            fa = f(a)
            fb = f(b)
            fc = f(c)
            x = -(a*fb*fc*(fb-fc)+b*fc*fa*(fc-fa)+c*fa*fb*(fa-fb))/((fa-fb)*(fb-fc)*(fc-fa))
            fx = f(x)
        flops += 22
        i += 1
    return x, i, flops

### 3. Metoda siecznych

In [9]:
def secant(f,a,b):
    i = 0
    flops = 5
    x_k1 = a
    fxk1 = f(x_k1)
    fb=f(b)
    if fxk1*fb>0:
        print('Funkcja w punktach wytyczających cięciwe ma ten same znak')
    x_k = b - fb*(b-x_k1)/(f(b)-fxk1)
    fx = f(x_k)
    while not np.isclose(fx, 0, rtol=1e-10):
        x_k, x_k1 = x_k - fx*(x_k-x_k1)/(fx-fxk1), x_k
        fx, fxk1 = f(x_k), f(x_k1)       
        i += 1
        flops += 5 
    return x_k, i, flops

### 4. Metoda Newtona 

In [10]:
def f_prim(x):
    return -1 - (1/np.cos(x)**2)

def newton(f,b):
    i = 0
    flops = 2
    x = b-f(b)/f_prim(b)
    fx = f(x)
    while not np.isclose(fx, 0):
        b=x
        x = b-f(b)/f_prim(b)
        fx = f(x)
        i += 1
        flops += 2
        if i > 1000:
            raise ValueError("Metoda Newtona nie zbiega do rozwiązania z podanego punktu startowego.")
    return x, i, flops

## Obliczanie całek

### 1. Wzór simpsona

In [11]:
def simpson(f, a, b, nodes):
    n = nodes-1
    h = (b-a)/n
    x0 = f(a)
    xn = f(b)
    xi = 0
    xj = 0
    for i in range(1, n):
        x = a + i*h
        if i%2 ==0:
            xi += f(x)
        else:
            xj += f(x)
    integral = h*(x0 + 4*xj + 2*xi + xn)/3
    return integral

### 2. Wzór trapezów

In [12]:
def trapezoidal(f, a, b, nodes):
    n = nodes-1
    h = (b-a)/n
    x0 = f(a)
    xn = f(b)
    mid = 0
    for i in range(1, n):
        x = a+i*h
        mid += f(x) 
    integral = h*((xn+x0)/2 + mid)
    return integral


### 3. Metoda Gaussa-Legendre’a

In [13]:
def GaussLegendre(f, a, b, weights, nodes):
    n = len(nodes)
    h = (b-a)/2
    c = (a+b)/2
    val = 0
    for i in range(0, n):
        val += weights[i]*f(h*nodes[i]+c)
    integral = h*val
    return integral

## Równania różniczkowe

### 1. Metoda Eulera

In [14]:
def euler(f, x0, y0, xStop, n):
    h = (xStop-x0)/n
    xs = [x0]
    ys = [y0]
    for i in range(n):
        ys.append(ys[-1]+h*f(xs[-1], ys[-1]))
        xs.append(xs[-1]+h)
    return xs, ys

### 2. Metoda Runge-Kutta drugiego rzędu

In [15]:
def soRungeKutta(f, x0, y0, xStop, n):
    h = (xStop-x0)/n
    xs = [x0]
    ys = [y0]
    for i in range(n):
        k1 = h*f(xs[-1], ys[-1])
        k2 = h*f(xs[-1]+h/2, ys[-1]+k1/2)
        ys.append(ys[-1]+k2)
        xs.append(xs[-1]+h)
    return xs, ys

### 3. Metoda Runge-Kutty czwartego rzędu

In [16]:
def foRungeKutta(f, x0, y0, xStop, n):
    h = (xStop-x0)/n
    xs = [x0]
    ys = [y0]
    for i in range(n):
        k1 = f(xs[-1], ys[-1])
        k2 = f(xs[-1]+h/2, ys[-1]+h*k1/2)
        k3 = f(xs[-1]+h/2, ys[-1]+h*k2/2)
        k4 = f(xs[-1]+h, ys[-1]+h*k3)
        ys.append(ys[-1] + h*(k1+2*k2+2*k3+k4)/6)
        xs.append(xs[-1]+h)
    return xs, ys
